In [1]:
from spinbox import *
from itertools import count
seeder = count(0,1) # generates a list of counting numbers for rng "seeds" (not really seeds in Numpy, but indices of seeds)

# The `spinbox` tutorial
## Part 2 : Simple calculations in the non-entangled space using product states

The classes `ProductState` and `ProductOperator` are analogous to `HilbertState` and `HilbertOperator` but they are individually restricted to tensor products of one-body states and one-body operators respectively.

To begin, let's do the same basic calculations we did before. A `ProductState` may be instantiated like so.

In [2]:
ket_1 = ProductState(n_particles=2, ketwise=True, isospin=False).randomize(seed=next(seeder))
print(ket_1)

ProductState ket of 2 particles: 
ket #0:
[[ 0.18722951-0.79768503j]
 [-0.19672223+0.53846454j]]
ket #1:
[[0.36861193+0.75055118j]
 [0.06037799+0.54511711j]]



Note that our usual matrix representation is replaced with just the one-body vectors. You can project a `ProductState` into a `HilbertState` by the `.to_manybody_basis()` method.

In [3]:
print(ket_1.to_manybody_basis())

HilbertState ket of 2 particles:
[[ 0.66771848-0.15351089j]
 [ 0.4461363 +0.05389939j]
 [-0.47665936+0.05083436j]
 [-0.30540393-0.07472524j]]


Addition and subtraction are not allowed for `Product` objects, because in general doing so will not product another `Product` object. 
We do, however, have scalar multiplication. There are two built-in ways to do it.

For a product state $|s_1\rangle \otimes |s_2\rangle$ multiplied by a scalar $c$,

$$c (|s_1\rangle \otimes |s_2\rangle) = c|s_1\rangle \otimes |s_2\rangle = |s_1\rangle \otimes c |s_2\rangle = c^{1/A}|s_1\rangle \otimes c^{1/A}|s_2\rangle$$

We can either choose a particle to multiply by $c$, or we multiply all of them by $c^{1/A}$.

In [4]:
c = 1000.0
print("Multiplying the first particle:")
print(ket_1.scale_one(0,c))
print("Multiplying the second particle:")
print(ket_1.scale_one(1,c))
print("Multiplying all particles:")
print(ket_1.scale_all(c))

Multiplying the first particle:
ProductState ket of 2 particles: 
ket #0:
[[ 187.22951251-797.68503332j]
 [-196.72222747+538.46454152j]]
ket #1:
[[0.36861193+0.75055118j]
 [0.06037799+0.54511711j]]

Multiplying the second particle:
ProductState ket of 2 particles: 
ket #0:
[[ 0.18722951-0.79768503j]
 [-0.19672223+0.53846454j]]
ket #1:
[[368.61193301+750.55118201j]
 [ 60.37799402+545.11711023j]]

Multiplying all particles:
ProductState ket of 2 particles: 
ket #0:
[[ 5.92071705-25.22501561j]
 [-6.22090305+17.0277439j ]]
ket #1:
[[11.65653281+23.73451236j]
 [ 1.90931982+17.2381166j ]]



Multiplication is handled in the same way as for `HilbertState`.

In [5]:
bra_2 = ProductState(n_particles=2, ketwise=False, isospin=False).randomize(seed=next(seeder))

print("< state 1 | state 1 > = " , ket_1.dagger() * ket_1)
print("< state 2 | state 1 > = " , bra_2 * ket_1)
print("| state 1 > < state 2 | = " , ket_1 * bra_2)

< state 1 | state 1 > =  (0.9999999999999999+0j)
< state 2 | state 1 > =  (0.0976436722152378-0.0992577246900124j)
| state 1 > < state 2 | =  ProductOperator
Op 0 Re:
[[ 0.58434323  0.37864789]
 [-0.41250184 -0.29851431]]
Op 0 Im:
[[-0.07883252 -0.42462983]
 [ 0.00592682  0.26332491]]
Op 1 Re:
[[ 0.33643132 -0.58753285]
 [ 0.20042615 -0.25350879]]
Op 1 Im:
[[ 0.03210537 -0.48968377]
 [ 0.09468723 -0.43289161]]



Likewise, the outer product has produced a `ProductOperator` object. The `ProductOperator` behaves analogous to the `HilbertOperator`.

In [6]:
operator = ProductOperator(n_particles=2, isospin=False).apply_sigma(particle_index=0, dimension=0) # sigma x on particle 1
print(operator)

ProductOperator
Op 0 Re:
[[0. 1.]
 [1. 0.]]
Op 0 Im:
[[0. 0.]
 [0. 0.]]
Op 1 Re:
[[1. 0.]
 [0. 1.]]
Op 1 Im:
[[0. 0.]
 [0. 0.]]



*Example*: Let's use the `ProductState` and `ProductOperator` classes to show $\langle \vec{\sigma}_1 \cdot \vec{\sigma}_2 \rangle_\psi$ is equal to $ \langle 2P_{12} - 1 \rangle_\psi$ for a random state $|\psi\rangle$ and the spin exchange operator $P_{12}$.

This is a good example calculation because we immediately see the difficulty presented by the sum of operations: $\hat{\sigma}_{1x}\hat{\sigma}_{2x} + \hat{\sigma}_{1y}\hat{\sigma}_{2y} + \hat{\sigma}_{1z}\hat{\sigma}_{2z} + I$. The best way to compute this is to break it into individual brackets, then add those together.

$$ \langle \psi | \hat{\sigma}_{1x}\hat{\sigma}_{2x}| \psi \rangle + \langle \psi |\hat{\sigma}_{1y}\hat{\sigma}_{2y}| \psi \rangle + \langle \psi |\hat{\sigma}_{1z}\hat{\sigma}_{2z}| \psi \rangle = 2 \langle \psi | P_{12} | \psi \rangle + \langle \psi | \psi \rangle$$

The `ProductState` class has a built-in exchange method, which I use here.

In [10]:
# states
psi = ProductState(n_particles=2, ketwise=True, isospin=False).randomize(seed=next(seeder))

# left hand side
lhs = 0.
for dimension in (0,1,2):
    op = ProductOperator(2, isospin=False).apply_sigma(0,dimension).apply_sigma(1,dimension)
    lhs += psi.dagger() * op * psi
    
# right hand side
rhs = 2 * (psi.dagger() * psi.exchange(0,1)) - 1
print("< psi | sigma_i . sigma_j | psi > = ", lhs)
print("2 < psi | P_ij | psi > - 1 = ", rhs)


< psi | sigma_i . sigma_j | psi > =  (0.9118851662186851+0j)
2 < psi | P_ij | psi > - 1 =  (0.9118851662186853+0j)
